# Multiple object tracking

In this note, we will explore a few of the most popular algorithms for multiple object tracking. 
We will start with a brief introduction to the problem, including the similarities and differences between object detection and object tracking. 
Building on this foundational understanding, we will introduce some of the most popular trackers, along with some libraries and repositories with implementations of these trackers.  



## Background
- Object detection vs. object tracking
    - Detections can vary from frame to frame due to blur, pose, occlusion, etc.
    - One object may recieve many tracked ids if a detection is not produced in a (set of) frame(s)
    - Tracking requires then both detections but also a way to associate them across frames
    - Tracking can be online or offline, depending on the computational resources available
    - Modern tracking techniques typically use a detector (YOLO, ResNet, etc.) together with a tracker (SORT, ByteTrack, etc.)
- Movement predictions require info about velocity, direction, etc.
    - Can be constant, determined beforehand or learned from data
    - Can be adaptive, as seen in Kalman filters 
- Some classic methods exploit derivative-like estimatations of postion from changes in detections between frames
    - Even simplier ones just look at IOU between objects found through filtering
- Some trackers that leverage deep learning incorporate visual information about each detected object to help performance
    - This can be done through embeddings, which are learned representations of the object's appearance
    - These embeddings can be used to associate objects across frames, even when they are occluded or change appearance
- Evaluating trackers is a difficult task, as there is no single metric that can capture all the nuances of tracking performance
    - MOTA, MOTP, etc. are some of the most popular metrics used to evaluate trackers
    - These metrics are based on the number of false positives, false negatives, and identity switches that occur during tracking
    - The MOTChallenge is a popular benchmark for evaluating multiple object tracking algorithms


# Tracking algorithims

## Classic
- Background detection
- Contour filtering
- Mean shift
- Particle filter
- Kalman filter (SORT)
- Hungarian algorithm (SORT)

## Modern
- DeepSORT
    - replace IOU with Mahalanobis distance
    - use embeddings to associate objects across frames
- StrongSORT
    - ...?
- ByteTrack
    - ...?
